In [1]:
!wget https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv

--2024-10-31 22:10:20--  https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv
Resolving github.com (github.com)... 64:ff9b::14cd:f3a6, 20.205.243.166
Connecting to github.com (github.com)|64:ff9b::14cd:f3a6|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv [following]
--2024-10-31 22:10:20--  https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 64:ff9b::b9c7:6c85, 64:ff9b::b9c7:6e85, 64:ff9b::b9c7:6f85, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|64:ff9b::b9c7:6c85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391501 (382K) [text/plain]
Saving to: ‘jamb_exam_results.csv’

jamb_exam_results.c 100%[===================>] 382.33K  90.1KB/s    in 4.2s    

2024-10-31 

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

In [38]:
df = pd.read_csv('jamb_exam_results.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head()

,jamb_score,study_hours_per_week,attendance_rate,teacher_quality,distance_to_school,school_type,school_location,extra_tutorials,access_to_learning_materials,parent_involvement,it_knowledge,student_id,age,gender,socioeconomic_status,parent_education_level,assignments_completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,1,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,2,15,Male,High,NaN,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,3,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,4,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,5,22,Female,Medium,Tertiary,1


### Preparation
- Remove the student_id column.
- Fill missing values with zeros.
- Do train/validation/test split with 60%/20%/20% distribution.
- Use the train_test_split function and set the random_state parameter to 1.
- Use DictVectorizer(sparse=True) to turn the dataframes into matrices.

In [39]:
df = df.drop(columns=['student_id'])

df = df.fillna(0)

In [47]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = np.log1p(df_train.jamb_score.values)
y_val = np.log1p(df_val.jamb_score.values)
y_test = np.log1p(df_test.jamb_score.values)

# df_train = df_train.drop(columns=['assignments_completed'])
# df_val = df_val.drop(columns=['assignments_completed'])
# df_test = df_test.drop(columns=['assignments_completed'])

# dv = DictVectorizer(sparse=True)

# X_train = dv.fit_transform(df_train.to_dict(orient='records'))
# X_val = dv.transform(df_val.to_dict(orient='records'))
# X_test = dv.transform(df_test.to_dict(orient='records'))
y_train


array([4.98360662, 5.44673737, 5.08759634, ..., 5.37989735, 5.45958551,
       4.6443909 ])

In [20]:
print(f'X_train shape: {X_train.shape}')
print(f'X_val shape: {X_val.shape}')
print(f'X_test shape: {X_test.shape}')

X_train shape: (3000, 29)
X_val shape: (1000, 29)
X_test shape: (1000, 29)


## Question 1
Let's train a decision tree regressor to predict the jamb_score variable.

Train a model with max_depth=1.
Which feature is used for splitting the data?

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.tree import export_text

model = DecisionTreeClassifier(max_depth=1)
model.fit(X_train, y_train)

print(export_text(model, feature_names=dv.get_feature_names_out()))

|--- study_hours_per_week <= 19.50
|   |--- class: 1
|--- study_hours_per_week >  19.50
|   |--- class: 2



In [32]:
y_val

array([1, 1, 2, 1, 3, 1, 3, 1, 1, 4, 4, 2, 1, 1, 1, 1, 3, 1, 2, 1, 1, 2,
       1, 2, 4, 2, 1, 2, 1, 1, 1, 2, 1, 3, 4, 1, 1, 1, 1, 1, 3, 1, 2, 3,
       1, 2, 3, 1, 2, 1, 1, 3, 1, 3, 3, 4, 1, 2, 2, 1, 3, 1, 1, 1, 2, 1,
       2, 2, 2, 1, 2, 2, 1, 1, 1, 2, 3, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 3, 1, 2, 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, 3, 1, 1, 3, 1,
       1, 1, 3, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 3, 2, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 3, 1, 2, 3, 3, 1, 2, 2, 2, 3, 1, 1, 1, 3, 3, 1, 1,
       1, 3, 2, 2, 2, 1, 3, 1, 2, 1, 1, 2, 2, 1, 1, 3, 2, 1, 2, 2, 3, 1,
       2, 1, 2, 2, 1, 5, 3, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1,
       1, 3, 1, 2, 2, 2, 1, 4, 1, 1, 1, 1, 4, 3, 3, 1, 1, 3, 3, 3, 1, 1,
       2, 3, 1, 5, 2, 3, 1, 1, 3, 2, 1, 2, 1, 3, 1, 1, 2, 2, 3, 1, 4, 2,
       1, 2, 4, 1, 1, 2, 2, 1, 3, 1, 2, 1, 1, 1, 1, 3, 1, 2, 3, 1, 4, 3,
       2, 1, 3, 2, 1, 3, 5, 2, 1, 1, 1, 3, 1, 2, 2, 4, 1, 2, 1, 1, 1, 3,
       2, 1, 5, 5, 1, 2, 4, 2, 1, 2, 2, 1, 4, 3, 1,

In [33]:
y_pred

array([0.35842541, 0.18556701, 0.35842541, 0.18556701, 0.35842541,
       0.35842541, 0.35842541, 0.18556701, 0.18556701, 0.35842541,
       0.35842541, 0.35842541, 0.18556701, 0.35842541, 0.18556701,
       0.18556701, 0.18556701, 0.18556701, 0.18556701, 0.18556701,
       0.35842541, 0.18556701, 0.35842541, 0.35842541, 0.35842541,
       0.35842541, 0.18556701, 0.18556701, 0.18556701, 0.35842541,
       0.35842541, 0.35842541, 0.18556701, 0.35842541, 0.35842541,
       0.18556701, 0.18556701, 0.35842541, 0.18556701, 0.18556701,
       0.18556701, 0.18556701, 0.35842541, 0.18556701, 0.18556701,
       0.35842541, 0.35842541, 0.35842541, 0.18556701, 0.35842541,
       0.18556701, 0.18556701, 0.18556701, 0.35842541, 0.35842541,
       0.35842541, 0.35842541, 0.18556701, 0.18556701, 0.18556701,
       0.35842541, 0.35842541, 0.18556701, 0.18556701, 0.18556701,
       0.35842541, 0.35842541, 0.18556701, 0.35842541, 0.35842541,
       0.35842541, 0.18556701, 0.35842541, 0.18556701, 0.35842

AxisError: axis 1 is out of bounds for array of dimension 1